In [7]:
from typing import TypedDict, Optional, Dict,Annotated
from langgraph.graph import StateGraph ,START,END
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langgraph.graph.message import  add_messages
from pydantic import  BaseModel,Field

In [8]:
class BuisnessNeedPreProcessed(BaseModel):
    core_functionies: Annotated[str, Field(description="Provide a detailed evaluation of the research significance, including its impact on the field and any novel contributions (4-5 sentences)")]
    

In [10]:
class CoddState:
    business_need:str
    buisness_need_pre_processed:BuisnessNeedPreProcessed
    
    pass

In [4]:
class State(TypedDict):
    messages:Annotated[list,add_messages]


graph_builder= StateGraph(State)